In [11]:
pip install langchain-community


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\quint\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
from pprint import pprint
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import FastEmbedSparse
from qdrant_client.models import  SparseVector
from qdrant_client import QdrantClient, models
from langchain import PromptTemplate    
from langchain.schema import BaseMemory
from langchain.memory import ChatMessageHistory

# from langchain.chains import RunnableWithMessageHistory
import numpy as np

from PyPDF2 import PdfReader
from PyPDF2 import PdfReader
import re
import os
import json
import openai
from pprint import pprint
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from dotenv import load_dotenv
from langchain.schema import Document
import os
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain.schema import Document
from fastembed import LateInteractionTextEmbedding
from colbert import Indexer
from colbert.infra import Run, RunConfig, ColBERTConfig
from ragatouille import RAGPretrainedModel
from qdrant_client.models import PointStruct, SparseVector
from qdrant_client.models import VectorParams, Distance
from qdrant_client import QdrantClient, models
from langchain_core.documents import Document
from pprint import pprint


In [22]:
from langchain_community.document_loaders import DirectoryLoader


In [82]:
def extract_text(file_path, start_after, stop_at, stop_at_min_page=10):
    reader = PdfReader(file_path)
    text = ""
    has_started = False

    for i, page in enumerate(reader.pages):
        page_text = page.extract_text()

        # Skip unreadable or empty pages
        if not page_text or page_text.strip() == "":
            continue

        # Start extracting after detecting the start phrase
        if not has_started and start_after in page_text:
            has_started = True
            continue

        # Stop processing at "Annex B" but only if it's after the defined minimum page
        if has_started and stop_at in page_text and i >= stop_at_min_page:
            break

        # Append text if extraction has started
        if has_started:
            text += page_text

    return text


def clean_and_split_text(text):
    # Remove license and metadata
    text = re.sub(r'Licensed to .*?prohibited\.', '', text, flags=re.DOTALL)
    text = re.sub(r'© ISO \d{4}.*?\n', '', text)
    text = re.sub(r'© ISO/IEC \d{4}.*?\n', '', text)


    # # Normalize spaces but preserve newlines for sections
    text = re.sub(r'\s{2,}', ' ', text)  # Replace excessive spaces with one
    text = re.sub(r'(\d+\.\d+)\s([A-Z])', r'\1\n\2', text)  # Ensure newlines after numbered sections
    text = re.sub(r'(\d+\sContext)', r'\n\1', text)  # Add newlines before "Context" sections
    return text



In [83]:
def chunk_ISO_sections(pdf_files, start_phrase, stop_phrase):

    


    all_chunks = {}

    for file in pdf_files:
        raw_text = extract_text(file, start_phrase, stop_phrase)
        
        cleaned_text = clean_and_split_text(raw_text)
        # print("test", file, cleaned_text)
        # print(cleaned_text)
        all_chunks[file] = cleaned_text

    # # Save to JSON for further processing
    # with open("Sections ISO\\ISOsubsectionsAll.json", "w") as outfile:
    #     json.dump(all_chunks, outfile, indent=4)

    # print("Sections ISO\\AllisoSUBS.json")
    
    return all_chunks
    

In [84]:
folder_path = "ISO standards"
pdf_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".pdf")]
print(pdf_files)

for file in pdf_files:
    print(file)

['ISO standards\\ISO_14001_2015(en).pdf', 'ISO standards\\ISO_50001_2018(en).pdf', 'ISO standards\\ISO_IEC_30134-1_2016(en).pdf']
ISO standards\ISO_14001_2015(en).pdf
ISO standards\ISO_50001_2018(en).pdf
ISO standards\ISO_IEC_30134-1_2016(en).pdf


In [85]:

start_phrase = "ISO (the International Organization for Standardization) is a worldwide federation"
stop_phrase = "Annex B"
text = chunk_ISO_sections(pdf_files, start_phrase, stop_phrase)
pprint(text)

{'ISO standards\\ISO_14001_2015(en).pdf': ' ISO 14001:2015(E)\n'
                                          'Introduction\n'
                                          '0.1\n'
                                          'Background\n'
                                          'Achieving a balance between the '
                                          'environment, society and the '
                                          'economy is considered essential to '
                                          'meet the needs of the present '
                                          'without compromising the ability of '
                                          'future generations to meet their '
                                          'needs. Sustainable development as a '
                                          'goal is achieved by balancing the '
                                          'three pillars of sustainability.\n'
                                          'Societal expectations 

In [26]:
# Load environment variables from .env file
load_dotenv()

# Access OpenAI API Key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Debug: Check if the API key is being correctly loaded
if openai_api_key is None or openai_api_key == "":
    print("OpenAI API Key not found! Please make sure it is set correctly in the .env file.")
else:
    print(f"OpenAI API Key Loaded: {openai_api_key[:4]}...")  # Only print part of the key for security reasons

OpenAI API Key Loaded: sk-p...


In [87]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [90]:
documents = [
    Document(page_content=content, metadata={"source": file_path})
    for file_path, content in text.items()
]

In [92]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(documents, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/3 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/3 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/3 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/31 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/63 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [93]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wht is the purpse of an Internationl Standrd i...,[Introduction 0.1 Background Achieving a balan...,The purpose of this International Standard is ...,single_hop_specifc_query_synthesizer
1,How does the Plan-Do-Check-Act (PDCA) model fa...,[Plan-Do-Check-Act model The basis for the app...,The Plan-Do-Check-Act (PDCA) model facilitates...,single_hop_specifc_query_synthesizer
2,What does ISO Guide 73:2009 define as 'risk'?,[part of the organization. 3.1.6 interested pa...,ISO Guide 73:2009 defines 'risk' as the effect...,single_hop_specifc_query_synthesizer
3,what be the role of international standard in ...,[entry: An internal audit is conducted by the ...,The International Standard provides the requir...,single_hop_specifc_query_synthesizer
4,How do energy performance indicators (EnPIs) a...,[<1-hop>\n\nISO 50001:2018(E) — clarification ...,Energy performance indicators (EnPIs) and key ...,multi_hop_abstract_query_synthesizer
5,How does the ISO/IEC 30134 series contribute t...,[<1-hop>\n\nISO/IEC 30134-1:2016(E) Introducti...,The ISO/IEC 30134 series contributes to enviro...,multi_hop_abstract_query_synthesizer
6,How does the implementation of an energy manag...,[<1-hop>\n\nISO/IEC 30134-1:2016(E) Introducti...,The implementation of an energy management sys...,multi_hop_abstract_query_synthesizer
7,How does the Plan-Do-Check-Act (PDCA) model in...,[<1-hop>\n\nISO 50001:2018(E) — clarification ...,The Plan-Do-Check-Act (PDCA) model is a founda...,multi_hop_abstract_query_synthesizer
8,How does ISO 50001:2018 ensure compatibility w...,[<1-hop>\n\nCompatibility with other managemen...,ISO 50001:2018 ensures compatibility with othe...,multi_hop_specific_query_synthesizer
9,How does the management of SEUs contribute to ...,[<1-hop>\n\nuse of more efficient energy using...,The management of SEUs (Significant Energy Use...,multi_hop_specific_query_synthesizer


In [94]:
import os
os.environ["RAGAS_APP_TOKEN"] = "apt.4475-5263bbe7f1bb-033d-9b03-1ae6d1f2-7b15f"
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/5f2930e1-e1d5-4145-b72e-0149d7de9352


'https://app.ragas.io/dashboard/alignment/testset/5f2930e1-e1d5-4145-b72e-0149d7de9352'

In [97]:
df = dataset.to_pandas()
df.to_csv('RAGAs_test_questions.csv')